## Step 3

In [127]:
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Make sure that the dataset df_join has 10 boroughs and 103 neighborhoods:

In [128]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_join['Borough'].unique()),
        df_join.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Define the geograpical coordinate of Toronto, TO 

In [133]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of New York with neighborhoods superimposed on top

In [134]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_join['Latitude'], df_join['Longitude'], df_join['Borough'], df_join['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Simplify the above map and segment and cluster only the neighborhoods in Scarborough. Slice the df_join and create a new dataframe of the Scarborough data:

In [135]:
sc_data = df_join[df_join['Borough'] == 'Scarborough'].reset_index(drop=True)
sc_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


Define the geograpical coordinate of Scarborough:

In [137]:
address = 'Scarborough, TO'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [139]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(sc_data['Latitude'], sc_data['Longitude'], sc_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Start explore the neighborhoods with Foursquare API...

Define Foursquare Credentials and Version:

In [140]:
CLIENT_ID = 'QZXLMNDBTU4E2BSECEKXZRPO4B3XMMYK4JCWEFFLAX0ACEWR' # Foursquare ID
CLIENT_SECRET = 'UJ41VCMULLRDORV1UNJBMO5SJXICRAZAMKZBFX2B3W4FYAOE' # Foursquare Secret
VERSION = '20200317' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: QZXLMNDBTU4E2BSECEKXZRPO4B3XMMYK4JCWEFFLAX0ACEWR
CLIENT_SECRET:UJ41VCMULLRDORV1UNJBMO5SJXICRAZAMKZBFX2B3W4FYAOE


### Explore the random neighborhood in our dataframe

In [146]:
sc_data.loc[3, 'Neighborhood']

'Woburn'

Get the neighborhood's latitude and longitude values:

In [148]:
neigh_latitude = sc_data.loc[3, 'Latitude'] # neighborhood latitude value
neigh_longitude = sc_data.loc[3, 'Longitude'] # neighborhood longitude value

neigh_name = sc_data.loc[3, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_latitude, 
                                                               neigh_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


### Get the top 100 venues that are in Woburn within a radius of 500 meters

Create the GET request URL:

In [150]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_latitude, 
    neigh_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QZXLMNDBTU4E2BSECEKXZRPO4B3XMMYK4JCWEFFLAX0ACEWR&client_secret=UJ41VCMULLRDORV1UNJBMO5SJXICRAZAMKZBFX2B3W4FYAOE&v=20200317&ll=43.7709921,-79.21691740000001&radius=500&limit=100'

Send the GET request and obtain the resutls

In [151]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e7003e3c546f3001bd33c13'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7754921045, 'lng': -79.21069729639068},
   'sw': {'lat': 43.7664920955, 'lng': -79.22313750360935}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cc1d28c06c254815ac18547',
       'name': 'Starbucks',
       'location': {'address': '300 Borough Dr',
        'crossStreet': 'Scarborough Town Centre',
        'lat': 43.770037201625215,
        'lng': -79.22115586641958,
        'labeledLatLngs':

All the information is in the items key. Set **get_category_type** function:

In [152]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a *pandas* dataframe:

In [156]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\VICTORY\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.770037,-79.221156
1,Tim Hortons,Coffee Shop,43.770827,-79.223078
2,Korean Grill House,Korean Restaurant,43.770812,-79.214502


How many venues were returned by Foursquare...

In [157]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## Explore Neighborhoods in Scarborough

### Create a function to repeat the same process to all the neighborhoods in Scarborough:

In [158]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Write the code to run the above function on each neighborhood and create a new dataframe called *sc_venues*

In [161]:
sc_venues = getNearbyVenues(names=sc_data['Neighborhood'],
                                   latitudes=sc_data['Latitude'],
                                   longitudes=sc_data['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


### Check the size of the resulting dataframe

In [162]:
print(sc_venues.shape)
sc_venues.head()

(86, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


Check how many venues were returned for each neighborhood:

In [164]:
sc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre, Wexford Heights",5,5,5,5,5,5
"East Birchmount Park, Ionview, Kennedy Park",6,6,6,6,6,6


### Find out how many unique categories can be curated from all the returned venues

In [165]:
print('There are {} uniques categories.'.format(len(sc_venues['Venue Category'].unique())))

There are 54 uniques categories.


## Analyze Each Neighborhood

In [166]:
# one hot encoding
sc_onehot = pd.get_dummies(sc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sc_onehot['Neighborhood'] = sc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sc_onehot.columns[-1]] + list(sc_onehot.columns[:-1])
sc_onehot = sc_onehot[fixed_columns]

sc_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Examine the new dataframe size:

In [168]:
sc_onehot.shape

(86, 55)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [169]:
sc_grouped = sc_onehot.groupby('Neighborhood').mean().reset_index()
sc_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.25,0.25,0.000,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.25,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.5,0.0,0.000000,0.000000,0.5,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
2,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.25,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
3,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.0,0.000000,0.00,0.125,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.125000,0.125000,0.00,0.000000,0.000000,0.125,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.125000,0.000000,0.0
4,"Clairlea, Golden Mile, Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.2,0.100000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.1,0.0,0.100,0.000000,0.000000,0.00,0.00,0.000,0.1,0.000,0.00,0.000000,0.000000,0.000000,0.1,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.00,0.1,0.000,0.000000,0.000000,0.000000,0.0
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000,0.000,0.083333,0.0,0.000000,0.0,0.000000,0.00,0.000,0.083333,0.000000,0.00,0.083333,0.000000,0.000000,0.000,0.083333,0.083333,0.083333,0.00,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.083333,0.000000,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000000,0.083333,0.0,0.0,0.083333,0.166667,0.0,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.083333,0.000000,0.0
6,"Cliffcrest, Cliffside, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.00,0.333333,0.333333,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.00,0.00,0.0,0.000,0.000000,0.000000,0.000000,0.0
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.0,0.4,0.000,0.000000,0.000000,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.2,0.000000,0.000

Confirm the new size:

In [170]:
sc_grouped.shape

(16, 55)

### Print each neighborhood along with the top 5 most common venues

In [171]:
num_top_venues = 5

for hood in sc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sc_grouped[sc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3            Pet Store   0.0
4    Korean Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0                   Café  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0  Fried Chicken Joint  0.12
1     Hakka Restaurant  0.12
2               Bakery  0.12
3                 Bank  0.12
4          Gas Station  0.12


----Clairlea, Golden Mile, Oakridge----
            venue  freq
0          Bakery   0.2
1        Bus Line   0.2

### Put that into a pandas dataframe

First, write a function to sort the venues in descending order:

In [172]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now, create the new dataframe and display the top 10 venues for each neighborhood:

In [175]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sc_grouped['Neighborhood']

for ind in np.arange(sc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sc_grouped.iloc[ind, :], num_top_venues)


In [213]:
neighborhoods_venues_sorted.tail()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,0,L'Amoreaux West,Chinese Restaurant,Fast Food Restaurant,Gym,Thrift / Vintage Store,Supermarket,Grocery Store,Coffee Shop,Breakfast Spot,Sandwich Place,Pizza Place
12,0,"Maryvale, Wexford",Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Department Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
13,2,"Rouge, Malvern",Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
14,3,Scarborough Village,Grocery Store,Playground,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
15,4,Woburn,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


# Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters:

In [257]:
# set number of clusters
kclusters = 6

sc_grouped_clustering = sc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 5, 4, 1, 1, 1, 1, 1, 1, 1])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood:

In [263]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


ValueError: cannot insert Cluster Labels, already exists

In [261]:
sc_merged = sc_data

In [262]:
# merge to add latitude/longitude for each neighborhood
sc_merged = sc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [270]:
sc_merged.dropna(axis = 0, inplace = True) #drop the row with NaN value
sc_merged.tail() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849,0,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Department Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,0,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
13,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter",43.781638,-79.304302,0,Pizza Place,Chinese Restaurant,Gas Station,Noodle House,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Thai Restaurant
14,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,3,Park,Playground,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
15,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Gym,Thrift / Vintage Store,Supermarket,Grocery Store,Coffee Shop,Breakfast Spot,Sandwich Place,Pizza Place


In [267]:
sc_merged['Cluster Labels'] = sc_merged['Cluster Labels'].astype('int') #change the type of 'Cluster Labels' to integer for creating map
sc_merged.dtypes

PostalCode                 object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels              int32
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

Visualize the resulting clusters:

In [268]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sc_merged['Latitude'], sc_merged['Longitude'], sc_merged['Neighborhood'], sc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster...

### Cluster_1

In [272]:
sc_merged.loc[sc_merged['Cluster Labels'] == 0, sc_merged.columns[[2] + list(range(5, sc_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",0,Intersection,Spa,Bank,Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Mexican Restaurant,Vietnamese Restaurant,Department Store
4,Cedarbrae,0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Department Store
6,"East Birchmount Park, Ionview, Kennedy Park",0,Department Store,Hobby Shop,Bus Station,Coffee Shop,Discount Store,Convenience Store,Vietnamese Restaurant,Hakka Restaurant,Gym,Grocery Store
7,"Clairlea, Golden Mile, Oakridge",0,Bus Line,Bakery,Park,Intersection,Metro Station,Bus Station,Ice Cream Shop,Soccer Field,Breakfast Spot,Electronics Store
8,"Cliffcrest, Cliffside, Scarborough Village West",0,American Restaurant,Motel,Movie Theater,Ice Cream Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,"Birch Cliff, Cliffside West",0,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,Gas Station
10,"Dorset Park, Scarborough Town Centre, Wexford ...",0,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Spa,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station
11,"Maryvale, Wexford",0,Bakery,Smoke Shop,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Department Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment
12,Agincourt,0,Latin American Restaurant,Breakfast Spot,Skating Rink,Lounge,Vietnamese Restaurant,Convenience Store,Gym,Grocery Store,General Entertainment,Gas Station
13,"Clarks Corners, Sullivan, Tam O'Shanter",0,Pizza Place,Chinese Restaurant,Gas Station,Noodle House,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Italian Restaurant,Bank,Thai Restaurant


### Cluster_2

In [273]:
sc_merged.loc[sc_merged['Cluster Labels'] == 1, sc_merged.columns[[2] + list(range(5, sc_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",1,Bar,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


### Cluster_3

In [274]:
sc_merged.loc[sc_merged['Cluster Labels'] == 2, sc_merged.columns[[2] + list(range(5, sc_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",2,Fast Food Restaurant,Vietnamese Restaurant,Convenience Store,Hobby Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


### Cluster_4

In [275]:
sc_merged.loc[sc_merged['Cluster Labels'] == 3, sc_merged.columns[[2] + list(range(5, sc_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,3,Grocery Store,Playground,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Gym,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
14,"Agincourt North, L'Amoreaux East, Milliken, St...",3,Park,Playground,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


### Cluster_5

In [276]:
sc_merged.loc[sc_merged['Cluster Labels'] == 4, sc_merged.columns[[2] + list(range(5, sc_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,4,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
